converte Listagens de rescisão e de férias recebidas em formato html para planilhas excell
registrando cada evento como uma linha

# Configuração do Ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import re
import html

# Pré Processamento

In [ ]:
def clean_html(arq_html):
  # Salva apenas as linhas que tem 'class'
  # e aplica a formatação para adicionar acentos
  arq_html = [html.unescape(row) for row in arq_html if 'class' in row]
  # Remove as divs
  for i, row in enumerate(arq_html):
    pattern = r'<[^>]*>'
    new_row = re.sub(pattern, '', row)
    arq_html[i] = new_row

  return arq_html

In [ ]:
def get_date(row):
  correspondencia = {
      '01/2023':   'Janeiro de 2023',
      '02/2023': 'Fevereiro de 2023',
      '03/2023':     'Março de 2023',
      '04/2023':     'Abril de 2023',
      '05/2023':      'Maio de 2023',
      '06/2023':     'Junho de 2023',
      '07/2023':     'Julho de 2023',
      '08/2023':    'Agosto de 2023',
      '09/2023':  'Setembro de 2023',
      '10/2023':   'Outubro de 2023',
      '11/2023':  'Novembro de 2023',
      '12/2023':  'Dezembro de 2023'
  }

  return correspondencia[row]

In [ ]:
def get_eventos(arq_html):
  eventos = list()
  registro = list()
  count = 0

  # Pode haver um 'Continua...' no meio dos eventos
  # então é necessário pular algumas linhas
  # A variável 'conteudo' indica se está sendo lido
  # algo a ser guardado
  # Ao encontrar uma linha que seja 'Continua...' o
  # valor de conteudo passa a ser falso, então os
  # valores das linhas serão ignorados até que seja
  # encontrada uma linha com 'Cargo', pois é onde
  # continuam os eventos
  conteudo = True
  for row in arq_html:
    if row == 'Continua...':
      conteudo = False
      registro = list()
      count = 0

    if conteudo:
      registro.append(row)
      count += 1
      if count == 4:
        eventos.append(registro)
        registro = list()
        count = 0
    else:
      if 'Cargo: ' in row:
        conteudo = True

  eventos = pd.DataFrame(eventos)

  if eventos.empty:
    return eventos

  # Larga a coluna de Referência
  eventos = eventos.drop(2, axis='columns')
  # Inverte a ordem das colunas
  eventos = eventos[eventos.columns[::-1]]

  codigos = list()
  descricoes = list()

  # Separa o código dos nomes
  for value in eventos[3]:
  # A coluna com nome 3 é a que estão os
  # códigos e descrições dos eventos
    value = value.split()
    codigos.append(value[0])
    desc = ' '.join(value[1:])
    descricoes.append(desc)
  # Adiciona os novos dados
  eventos.loc[:,'Código'] = codigos
  eventos.loc[:,'Descrição'] = descricoes
  # Atualiza o nome das colunas
  eventos.columns = ['Cód_Desc', 'Provento', 'Desconto', 'Código', 'Descrição']
  # Larga a coluna com código e descrição dos eventos
  eventos = eventos[['Código', 'Descrição', 'Provento', 'Desconto']]



  return eventos

In [ ]:
def get_registros_rescisao(path, nome):
  # Abre o arquivo html
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')

  # Remove informações desnecessárias do arquivo
  arq_html = clean_html(arq_html)

  columns = ['Competência', 'Empregador', 'Código_Empregado', 'Nome_Empregado', 'Cargo', 'Lotação', 'Código_Evento', 'Desc_Evento', 'Provento', 'Desconto']
  registros = pd.DataFrame(columns=columns)

  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  lotacao = str()
  cargo = str()
  start = int()
  end = int()

  # Encontra o empregador
  for i, row in enumerate(arq_html):
    if row == 'Empresa: ':
      empregador = arq_html[i+1]
      break
  # Tira o CNPJ do empregador
  index = empregador.find('- CNPJ')
  empregador = empregador[:index-1]

  # Encontra a lotação
  for i, row in enumerate(arq_html):
    if row == 'Empregado':
      lotacao = arq_html[i+1]
      break
  # Tira o código da lotação
  index = lotacao.find(' - ')
  lotacao = lotacao[index+3:]

  '''  # O começo dos dados são marcados pela palavra 'Empregado'
  for i, row in enumerate(arq_html):
    if row == 'Empregado':
      start = i+1
      break'''


  while True:

    # Encontra a competência
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Rescisão: ' in row:
        # O [16:] Remove o 'Rescisão: xx/' sendo xx o dia
        competencia = get_date(row[13:])
        # O index é guardado pois o código, nome e cargo
        # estão ao seu redor
        index = i
        break

    codigo_emp = arq_html[index-3]
    nome_emp = arq_html[index-2]
    cargo = arq_html[index+1]
    # O cargo começa com 'Cargo: '
    cargo = cargo[7:]

    # Coleta dos eventos

    # Começo dos eventos
    # Pode haver um 'Continua...' após o cargo
    if arq_html[index+2] == 'Continua...':
      # Neste caso os eventos só irão aparecer embaixo da próxima palavra 'Cargo'
      for i, row in enumerate(arq_html[index+2:], start=index+2):
        if 'Rescisão: ' in row:
          index = i
          break

    # Marco do fim dos eventos
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Admissão: ' in row:
        end = i-4
        break

    eventos = get_eventos(arq_html[index+2:end])

    # Salva um registro para cada evento
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, codigo_emp, nome_emp, cargo, lotacao]
      if eventos['Código'].iloc[i]:
        registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])
        registros.loc[len(registros)] = registro

    # Encontra a próxima pessoa

    for i, row in enumerate(arq_html[start:], start=start):
      if 'Pagamento:' in row:
        start = i+1
        break
    # Verifica se acabou
    if arq_html[start] == 'Total Geral':
      break

    # Verifica se a lotação é atualizada
    # As lotações aparecem após a linha que tem 'Pagamento: '
    # Porém pode ser já o código do próximo funcionário
    # Lotações tem ' - ' no nome, então serão indetificadas assim
    if ' - ' in arq_html[start]:
      lotacao = arq_html[start]
      index = lotacao.find(' - ')
      lotacao = lotacao[index+3:]
      start += 1

  nome = nome+'.xlsx'
  registros.to_excel(nome, index=False)

  #return registros

In [ ]:
def get_registros_ferias(path, nome):
  # Abre o arquivo html
  with open(path, 'r', encoding='latin-1') as file:
    arq_html = file.read()
  arq_html = arq_html.split('\n')

  # Remove informações desnecessárias do arquivo
  arq_html = clean_html(arq_html)

  columns = ['Competência', 'Empregador', 'Código_Empregado', 'Nome_Empregado', 'Cargo', 'Lotação', 'Código_Evento', 'Desc_Evento', 'Provento', 'Desconto']
  registros = pd.DataFrame(columns=columns)

  competencia = str()
  empregador = str()
  codigo_emp = str()
  nome_emp = str()
  lotacao = str()
  cargo = str()
  start = int()
  end = int()

  # Encontra o empregador
  for i, row in enumerate(arq_html):
    if row == 'Empresa: ':
      empregador = arq_html[i+1]
      break
  # Tira o CNPJ do empregador
  index = empregador.find('- CNPJ')
  empregador = empregador[:index-1]

  # Encontra a lotação
  for i, row in enumerate(arq_html):
    if row == 'Evento':
      lotacao = arq_html[i+1]
      break
  # Tira o código da lotação
  index = lotacao.find(' - ')
  lotacao = lotacao[index+3:]

  while True:

    # Encontra o cargo
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Cargo: ' in row:
        cargo = row[7:]
        index = i
        break
    codigo_emp = arq_html[index-2]
    nome_emp = arq_html[index-1]
    # Encontra a competência
    # A competência vem na forma 'Gozo: dd/mm/aaaa a dd/mm/aaaa'
    # Os meses podem ser diferentes, então será considerado o
    # primeiro mês como competência
    # Então só é necessários salvar os índices 9 a 15
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Gozo: ' in row:
        competencia = get_date(row[9:16])
        break

    for i, row in enumerate(arq_html[start:], start=start):
      if row == 'Líquido a receber:':
        end = i
        break

    eventos = get_eventos(arq_html[index+1:end])

    # Salva um registro para cada evento
    for i in range(eventos.shape[0]):
      registro = [competencia, empregador, codigo_emp, nome_emp, cargo, lotacao]
      if eventos['Código'].iloc[i]:
        registro.extend([eventos['Código'].iloc[i], eventos['Descrição'].iloc[i], eventos['Provento'].iloc[i], eventos['Desconto'].iloc[i]])
        registros.loc[len(registros)] = registro


    # Encontra o começo dos dados da próxima pessoa
    for i, row in enumerate(arq_html[start:], start=start):
      if 'Abono: ' in row:
        start = i+1
        break
    # Verifica se acabou
    if arq_html[start] == 'Total Geral':
      break
    # Verifica se a lotação foi atualizada
    if ' - ' in arq_html[start]:
      lotacao = arq_html[start]
      index = lotacao.find(' - ')
      lotacao = lotacao[index+3:]

  nome = nome+'.xlsx'
  registros.to_excel(nome, index=False)

  #return registros

#_
